In [1]:
import importlib

package_name = "datasets"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

datasets is already installed.


In [2]:
import importlib

package_name = "accelerate"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

accelerate is already installed.


In [3]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForPreTraining
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/6386"

# from colab.file_utils import load_required
# load_required(force_reinstall_pytorch=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/6386


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
import os
print(os.getcwd())

/content/drive/MyDrive/6386


In [7]:
# !pip install datasets

In [8]:
# # Hugging Face pre-trained
# tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base")
# model = AutoModelForPreTraining.from_pretrained("nlpaueb/sec-bert-base")

In [9]:
# checkpoint = "bert-base-uncased"
# checkpoint = "bert-base-cased"
checkpoint = "distilbert-base-uncased"
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.7.output.dense.weight', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.3.output.dense.weight', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.3.attention.self.value.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.3.attention.self.query.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.9.attention.self.value.bias', 'encoder.layer.10.attention.output.LayerNorm.weight', 'encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.4.attention.output.Laye

In [10]:
raw_datasets = load_from_disk("finetune_data")

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8
    })
    test_unlabeled: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [12]:
tokenized_datasets['train']['text'][:10]

["Both the net sales and operating profit were record high in the company 's history .",
 'RT @joemccann the correleation between the dollar index and $SPY is simply amazing. $DX finds support, $SPY retreats like clockwork',
 'Sales of security and system packaging increased slightly .',
 'NaturalGas Settles At 3-year Low $DBO $BNO  http://stks.co/d27qX',
 'Good Morrrning Retail!!! Todays Plays: $BBIG $PPSI $MARA $SOFI $SLDB $DIDI $AMC $BRDS Good Luck ð\x9f\x8d\x80',
 "18 May 2010 - Finnish electronics producer Elcoteq SE HEL : ELQAV said today that it has signed an extensive cooperation agreement on industrialisation , manufacturing , distribution and after-market services for mobile phones with Japan 's Sharp TYO : 6753 .",
 "$AAPL's CEO just made a major announcement. What's the impact on the stock? ????",
 'Now glancing at $SE which was once called the Amazon of SE Asia with a gaming arm. Gaming falling off hard as people go out more, but e-commerce and payments have good growth. I

In [13]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-13-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [14]:
!pip install transformers[torch]
!pip install accelerate -U

In [15]:
import accelerate
print(accelerate.__version__)

0.24.1


In [16]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to "epoch" to save both best and last models
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # disable wandb
    # fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.701206,0.500000,0.000000
2,No log,0.699938,0.500000,0.000000
3,No log,0.697644,0.500000,0.000000


TrainOutput(global_step=3, training_loss=0.7668245633443197, metrics={'train_runtime': 53.5981, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.056, 'total_flos': 764666504640.0, 'train_loss': 0.7668245633443197, 'epoch': 3.0})

In [18]:
output_dir = "saved_model"
os.makedirs(output_dir, exist_ok=True)

# Save the best model and the last model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [19]:
tokenized_datasets["test"]['text']

['Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts',
 'user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  ',
 "user I'd be afraid to short AMZN - they are looking like a near-monopoly in eBooks and infrastructure-as-a-service",
 'MNTA Over 12.00  ',
 'OI  Over 21.37  ',
 'PGNX  Over 3.04  ',
 'AAP - user if so then the current downtrend will break. Otherwise just a short-term correction in med-term downtrend.',
 "Monday's relative weakness. NYX WIN TIE TAP ICE INT BMC AON C CHK BIIB  "]

In [20]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(8, 2) (8,)


In [21]:
predictions

PredictionOutput(predictions=array([[-0.01037394, -0.23742616],
       [-0.0082894 , -0.3313913 ],
       [-0.05561402, -0.33699414],
       [ 0.07951535, -0.13008961],
       [ 0.08491139, -0.23141351],
       [ 0.07152186, -0.17986003],
       [-0.02059496, -0.27584192],
       [ 0.00868374, -0.2967829 ]], dtype=float32), label_ids=array([1, 1, 1, 1, 1, 1, 0, 0]), metrics={'test_loss': 0.7680109739303589, 'test_accuracy': 0.25, 'test_f1': 0.0, 'test_runtime': 0.1214, 'test_samples_per_second': 65.917, 'test_steps_per_second': 8.24})

In [22]:
probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.5565204  0.44347957]
 [0.58008003 0.41991997]
 [0.56988454 0.43011543]
 [0.5522102  0.44778976]
 [0.5784283  0.42157164]
 [0.5625166  0.4374834 ]
 [0.5634675  0.43653244]
 [0.57577837 0.42422166]]
Predicted Labels:
 [0 0 0 0 0 0 0 0]


In [23]:
true_labels = tokenized_datasets["test"]["label"]

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.25
Precision: 0.0
Recall: 0.25
F1 Score: 0.1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# helper_model.print_wrong_classifications(predicted_labels, true_labels, raw_datasets['test'])

Use the saved model to predit on the unlabled testing data.

In [26]:
# Define the path to the saved model directory
saved_model_dir = "saved_model"

# Load the saved tokenizer
tokenizer = BertTokenizer.from_pretrained(saved_model_dir)

# Load the saved model
model = BertForSequenceClassification.from_pretrained(saved_model_dir)
model

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [29]:
trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_datasets["test_unlabeled"])
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.58869964 0.41130036]
 [0.5823089  0.4176911 ]
 [0.5820477  0.41795227]
 [0.57150066 0.42849937]
 [0.5641721  0.4358279 ]
 [0.57707214 0.42292783]
 [0.5798263  0.42017373]
 [0.5812539  0.4187461 ]
 [0.56878144 0.43121853]
 [0.56459635 0.43540365]]
Predicted Labels:
 [0 0 0 0 0 0 0 0 0 0]


In [34]:
result_df = pd.DataFrame({
    'text': raw_datasets['test_unlabeled']['text'],
    'Predicted_Labels': predicted_labels,
    'Probability_Class_0': probabilities[:, 0],
    'Probability_Class_1': probabilities[:, 1], # 2 classes
})

output_csv_path = "predictions.csv"
result_df.to_csv(output_csv_path, index=False)
print(f"Predictions have been saved to {output_csv_path}")

Predictions have been saved to predictions.csv
